In [1]:
import numpy as np
import pandas as pd
import csv
import json
import time

from datetime import date, datetime, timedelta

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
class chk:
    def __init__(self, cin, cout):
        self.cin = cin
        self.cout = cout

In [8]:
def chk_inout():
    
    chk_list = []
    N_DAYS = 5
    STR_LEN = len('2019-02-09')
    
    t_date = datetime.now()
    
    for i in range(3,3+N_DAYS):
        
        chkin = (t_date + timedelta(days=i))
        chkout = (t_date + timedelta(days=i+1))
        
        chkin = chkin.strftime('%a %b %d %Y')
        chkout = chkout.strftime('%a %b %d %Y')
        
        chk_list.append(chk(chkin, chkout))

    return chk_list

In [9]:
chk_list = chk_inout()

for c in chk_list:
    print(c.cin, c.cout)

Sat Apr 06 2019 Sun Apr 07 2019
Sun Apr 07 2019 Mon Apr 08 2019
Mon Apr 08 2019 Tue Apr 09 2019
Tue Apr 09 2019 Wed Apr 10 2019
Wed Apr 10 2019 Thu Apr 11 2019


In [10]:
s_results = []

In [11]:
def get_search_url(chk_list):
    
    frmat = '%a %b %d %Y'
    
    macpath = '/Users/admin/Downloads/chromedriver'
    windows_path = 'E:\Downloads\Programs\chromedriver_win32\chromedriver.exe'
    
    searchbox_xpath = "//div[@class='IconBox IconBox--autocomplete']"
    textbox_xpath = "//input[@class='SearchBoxTextEditor SearchBoxTextEditor--autocomplete']"
    calendar_xpath = "//div[@class='IconBox IconBox--checkIn' and @data-element-name='check-in-box']"
    button_xpath = "//button[@class='btn Searchbox__searchButton Searchbox__searchButton--active']"
    
    sel_chkin_xpath = "//div[@class='DayPicker-Day DayPicker-Day--selected DayPicker-Day--checkIn']"
    sel_chkout_xpath = "//div[@class='DayPicker-Day DayPicker-Day--selected DayPicker-Day--checkOut']"
    sel_chkout_xpath_wkend = "//div[@class='DayPicker-Day DayPicker-Day--selected DayPicker-Day--checkOut DayPicker-Day--weekends']"
    
    for chk in chk_list:
        driver = webdriver.Chrome(windows_path)
        driver.get(base_url)

        assert "Agoda" in driver.title
        
        chkin = chk.cin
        chkout = chk.cout
        
        time.sleep(2)
        
        searchbox_elem = driver.find_element_by_xpath(searchbox_xpath).click()
        
        textbox_elem = driver.find_element_by_xpath(textbox_xpath)
        textbox_elem.send_keys("Bangkok")
        time.sleep(1)
        
        textbox_elem.send_keys(Keys.RETURN)
        time.sleep(1)
        
        sel_chkin_date_str = driver.find_element_by_xpath(sel_chkin_xpath).get_attribute('aria-label')
        sel_chkout_date_str = driver.find_element_by_xpath(sel_chkout_xpath).get_attribute('aria-label')
        
        chkin_date = datetime.strptime(chkin, frmat)
        chkout_date = datetime.strptime(chkout, frmat)
        
        sel_chkin_date = datetime.strptime(sel_chkin_date_str, frmat)
        sel_chkout_date = datetime.strptime(sel_chkout_date_str, frmat)
        
        tdelta = sel_chkin_date - chkin_date
        
        date_1_xpath = "//div[@class='DayPicker-Day' and @aria-label='" + chkin + "']"
        if(chkin[:3]=='Sat' or chkin[:3]=='Sun'):
            date_1_xpath = "//div[@class='DayPicker-Day DayPicker-Day--weekends' and @aria-label='" + chkin + "']"
        
        if(tdelta.days>0):
            date_2_xpath = "//div[@class='DayPicker-Day DayPicker-Day--selected' and @aria-label='" + chkout + "']"
            if(chkout[:3]=='Sat' or chkout[:3]=='Sun'):
                date_2_xpath = "//div[@class='DayPicker-Day DayPicker-Day--selected DayPicker-Day--weekends' and @aria-label='" + chkout + "']"
        
        elif(tdelta.days==0):
            date_1_xpath = sel_chkin_xpath
            date_2_xpath = sel_chkout_xpath
            
        elif(tdelta.days==-1):
            date_1_xpath = sel_chkout_xpath
        
        if(tdelta.days<0):
            date_2_xpath = sel_chkout_xpath
            if(chkout[:3]=='Sat' or chkout[:3]=='Sun'):
                date_2_xpath = sel_chkout_xpath_wkend
        
#         chkin_elem = driver.find_element_by_xpath(calendar_xpath).click()
#         time.sleep(2)

        chkin_date_elem = driver.find_element_by_xpath(date_1_xpath).click()
        time.sleep(1)

        chkout_date_elem = driver.find_element_by_xpath(date_2_xpath).click()
        
        button_elem = driver.find_element_by_xpath(button_xpath).click()

        search_result_url = driver.current_url
        s_results.append(search_result_url)
        
        time.sleep(1)
        driver.implicitly_wait(3)
        driver.close()
        
    return s_results

In [12]:
base_url = 'https://www.agoda.com/'
search_res_url = get_search_url(chk_list)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='DayPicker-Day DayPicker-Day--selected DayPicker-Day--checkOut']"}
  (Session info: chrome=73.0.3683.86)
  (Driver info: chromedriver=72.0.3626.69 (3c16f8a135abc0d4da2dff33804db79b849a7c38),platform=Windows NT 10.0.17134 x86_64)


In [ ]:
for url in search_res_url:
    print(url)
    print()

In [ ]:
with open('agoda-urls.txt','w') as f:
    for url in res_url:
        f.write(url + '\n')
    f.close()